## SQLite
### Exercices Chapitre I

In [ ]:
#On importe les package nécessaire à l'exercice.
import pandas as pd
import sqlite3
import tqdm
import random

#1. Read all the data
df = pd.read_csv("/Users/polo11/Downloads/bc.csv")
df

In [ ]:
print("nombre de ligne :", (df.shape[0]))
print("nombre de colonne :", (df.shape[1]))

In [ ]:
# 2: Insert the bc.csv in the sqlite format.
import sqlite3

conn = sqlite3.connect('/Users/polo11/Downloads/bc2.db')
c = conn.cursor()





In [ ]:
#3. In the same db create a new table called tomatch with two variables id_ and rdm_float.
# Insérez le DataFrame dans la table de la base de données SQLite
df.to_sql('bc2', conn, if_exists='replace', index=False)
#On crée la table tomatch
def create_table():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS tomatch(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
        conn.commit()
    except Exception as e:
        print(str(e))
        
create_table()

In [ ]:
# 4: In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.
query = """INSERT INTO tomatch(id_,rdm_float) VALUES (?,?)"""
list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,2)):
    values = (i, random.uniform(0,1))
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        conn.commit()
        list_of_insertion = []

In [ ]:
# 5: Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.

c.execute("""SELECT avg(rdm_float), count(rdm_float), max(rdm_float), min(rdm_float), sum(rdm_float) FROM bc2
             INNER JOIN tomatch ON tomatch.id_=bc2.id""")

docs = c.fetchall()
print(docs)

In [ ]:
# 6: How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

c.execute("SELECT COUNT(*) FROM bc2 WHERE radius_mean > 15")

count_radius_mean_gt_15 = c.fetchone()[0]

print("Radius moyen > 15 :", count_radius_mean_gt_15)

c.execute("SELECT COUNT(*) FROM bc2 WHERE radius_mean > 15 AND texture_mean > 20")

count_both_gt_15_20 = c.fetchone()[0]

print("Radius moyen > 15 et texture moyenne > 20 :", count_both_gt_15_20)

In [ ]:
# 7: For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups
# Exécuter une requête SQL pour calculer la moyenne de la variable souhaitée pour chaque catégorie de diagnostic
c.execute("""
          SELECT
              diagnosis,
              ROUND(AVG(radius_mean), 2) AS avg_radius_mean,  -- Moyenne arrondie de radius_mean avec 2 chiffres après la virgule
              ROUND(AVG(texture_mean), 2) AS avg_texture_mean   -- Moyenne arrondie de texture_mean avec 2 chiffres après la virgule
          FROM
              bc2
          GROUP BY
              diagnosis
          """)

# Récupérer les résultats de la requête
results = c.fetchall()

# Afficher les résultats
for row in results:
    print("Diagnostic:", row[0])
    print("moyenne radius_mean:", row[1])
    print("moyenne texture_mean:", row[2])
    print()

# Calculer la différence entre les moyennes des deux groupes
diff_avg_radius_mean = results[0][1] - results[1][1]
diff_avg_texture_mean = results[0][2] - results[1][2]

print("Différence de radius_mean moyen entre les 2 groupes:", diff_avg_radius_mean)
print("Différence de texture_mean moyenne entre les 2 groupes:", diff_avg_texture_mean)

In [ ]:
# 8. Change the name of the column "diagnosis" to "label"
c.execute("ALTER TABLE bc2 RENAME COLUMN diagnosis TO label")

conn.commit()

In [ ]:
# 9. Add a column named "day" of type DATE.
c.execute("ALTER TABLE bc2 ADD COLUMN day DATE")

conn.commit()

In [ ]:
# 10. Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)
c.execute("ALTER TABLE bc2 ADD COLUMN area_mean_int INTEGER")
conn.commit()
c.execute("UPDATE bc2 SET area_mean_int = CAST(area_mean AS INTEGER)")
conn.commit()

In [ ]:
# Histoire de s'assurer que tout cela a bien fonctionné :
c.execute("PRAGMA table_info(bc2)")

columns_info = c.fetchall()

for column_info in columns_info:
    print(column_info)

In [ ]:
#11. Create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.
c.execute("""CREATE TABLE IF NOT EXISTS tomatch2(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
conn.commit()

query = """INSERT INTO tomatch2(id_,rdm_float) VALUES (?,random())"""

list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,10)):
    values = (i,)
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        conn.commit()
        list_of_insertion = []

In [ ]:
# 12: Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2.
# Exécuter une requête SQL pour effectuer une jointure interne des tables "bc2", "tomatch" et "tomatch2" avec les conditions spécifiées
c.execute("""
          SELECT *
          FROM bc2
          INNER JOIN tomatch ON bc2.id = tomatch.id_
          INNER JOIN tomatch2 ON bc2.id = tomatch2.id_
          WHERE bc2.radius_mean > 15
          AND tomatch.rdm_float > 0.50
          AND tomatch2.rdm_float > 0.50
          """)

# Récupérer les résultats de la requête
results = c.fetchall()

# Afficher les résultats
for rows in results:
    print(rows)